<a href="https://colab.research.google.com/github/miguelamda/DL/blob/master/Ejercicios/Ejercicio2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 2. Entrenamiento de modelos de clasificación con Keras

En este segundo ejercicio tendrás que trabajar con Keras para probar las técnicas que has visto hasta la Práctica 5.2, incluyendo regularización (L2, dropout, data augmentation, K-validación cruzada, etc.), capas convolucionales, pooling, densas, etc. con **al menos 4 configuraciones** distintas. Además, emplearás tu modelo con tu webcam.

Para ello, vas a tener que trabajar con un conjunto de datos sobre imágenes de personas que llevan o no máscara. 

![mask](https://github.com/miguelamda/DL/blob/master/Ejercicios/img/mask.jpg?raw=1)

## 1. Enunciado

Empleando el dataset de mascarillas (ver apartado 3), debes construir y probar **al menos 4** configuraciones de modelos con Keras que sean sustancialmente distintas. Debes jugar con:
* K-validación o validación cruzada.
* Regularización: L1/L2, early stopping, dropout.
* Data augmentation: echando un vistazo al dataset, ¿qué tipo de transformación harías? ¿qué transformaciones NO harías? ¿qué ventaja se obtiene al usar aumentado?.
* Distintas arquitecturas de redes convolucionales: distinta cantidad y orden de capas convolucionales, pooling, densas, etc.
* Distintos optimizadores (Adam, RMSprop, momentum, etc.), factor de aprendizaje y número de épocas.

Después, entrénalas con los datos leídos y analiza los resultados, haciendo una comparativa y razonando lo obtenido.

## 2. Entrega

La entrega de este ejercicio se realiza a través de la tarea creada para tal efecto en Enseñanza Virtual. Tienes que entregar un notebook, y el HTML generado a partir de él, cuyas celdas estén ya evaluadas.

La estructura del notebook debe contener los siguientes apartados:

0. Cabecera: nombre y apellidos.
1. Dataset: descripción, carga y visualización.
2. Preparación de los datos para ser usados en Keras.
3. Modelos y configuraciones creados en Keras (un sub-apartado para cada uno, explicando de forma razonada, con tus palabras y figuras, la arquitectura probada).
4. Entrenamiento y evaluación de cada modelo creado (un sub-apartado para cada uno).
5. Análisis de resultados.
6. Bibliografía utilizada (enlaces web, material de clase, libros, etc.).

### 2.1. Nota importante
-----
**HONESTIDAD ACADÉMICA Y COPIAS: un trabajo práctico es un examen, por lo que
debe realizarse de manera individual. La discusión y el intercambio de
información de carácter general con los compañeros se permite (e incluso se
recomienda), pero NO AL NIVEL DE CÓDIGO. Igualmente el remitir código de
terceros, OBTENIDO A TRAVÉS DE LA RED o cualquier otro medio, se considerará
plagio.** 

**Cualquier plagio o compartición de código que se detecte significará
automáticamente la calificación de CERO EN LA ASIGNATURA para TODOS los
alumnos involucrados. Por tanto a estos alumnos NO se les conservará, para
futuras convocatorias, ninguna nota que hubiesen obtenido hasta el momento.
SIN PERJUICIO DE OTRAS MEDIDAS DE CARÁCTER DISCIPLINARIO QUE SE PUDIERAN
TOMAR.**

-----

## 3. El Dataset: Mask Dataset <a class="anchor" id="transferdata"></a>

Este pequeño dataset está disponible en Kaggle. Es una versión reducida del construido en [este tutorial](https://www.pyimagesearch.com/2020/05/04/covid-19-face-mask-detector-with-opencv-keras-tensorflow-and-deep-learning/). La pandemia originada por el SARS-Cov2 ha propiciado la creación de diversos datasets para detectar si las personas están llevando mascarilla o no. En este dataset para este ejeercicio, tenemos imágenes de personas con y sin mascarilla (algunas se les ha puesto la mascarilla de forma "artificial"). El enlace al dataset es el siguiente [Face Mask Detection](https://www.kaggle.com/omkargurav/face-mask-dataset). Si no tienes cuenta en Kaggle, o si quieres ir más rápido, puedes ejecutar la siguiente celda. Descargará el dataset directamente en tu sesión/ordenador desde un enlace directo.

In [ ]:
from tensorflow import keras
import pathlib

# Esta función de Keras descargará el dataset en $HOME/.keras
data_path = keras.utils.get_file(
    "mask_dataset.zip",
    "https://hdvirtual.us.es/discovirt/index.php/s/dijNRMPYfybYd3N/download",
    extract=True,
    archive_format='zip'
)

# La siguiente variable contiene la ruta al dataset. Puedes evaluarla
# y después hacer !ls para ver el contenido
DATA_FOLDER = pathlib.Path(data_path).parent / 'mask_ds' 
!ls $DATA_FOLDER

# Puedes usar el siguiente tamaño de imagen
IMAGE_SIZE = (128, 128)

# 4. Despliegue del modelo pre-entrenado

Una vez hayas entrenado un modelo (por ejemplo, `model`), puedes lanzar una de las siguientes celdas para hacer la prueba con tu WebCam. No es necesario que adjuntes capturas tuyas, pero sí que se valorará que hagas varias pruebas y analices el ratio de aciertos de tu modelo.

## 4.1 Capturando la WebCam desde Google Colab

Lanza las siguientes dos celdas para capturar una foto con tu web cam, si estás en Google Colab. Haz clic en el botón "Capture" (guardará la imagen en `image.jpg`).

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='image.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

##4.2 Capturando la WebCam desde local

El siguiente código debería permitirte capturar una foto desde tu webcam si estás en local.

In [ ]:
from IPython.display import HTML, display, update_display

main_text = """
<video id="video" width="320" height="240" autoplay style="display:none"></video>
<canvas id="canvas" width="320" height="240"  ></canvas>

<script>
// Grab elements, create settings, etc.
var video = document.getElementById('video');

// Get access to the camera!
if(navigator.mediaDevices && navigator.mediaDevices.getUserMedia) {
    // Not adding `{ audio: true }` since we only want video now
    navigator.mediaDevices.getUserMedia({ video: true }).then(function(stream) {
        //video.src = window.URL.createObjectURL(stream);
        //video.play();
        video.srcObject=stream;
        video.play();
    });
}

// Elements for taking the snapshot
var canvas = document.getElementById('canvas');
var context = canvas.getContext('2d');
var video = document.getElementById('video');


// Trigger photo take

setInterval(function() {
    context.drawImage(video, 0, 0, 320, 240);
    var myCanvas = document.getElementById('canvas');
    var image = myCanvas.toDataURL("image/png");
    IPython.notebook.kernel.execute("image = '" + image + "'")
    
}, 2);

</script>

"""
HTML(main_text)

#4.3 Evaluando el modelo

La siguiente celda debería permitirte cargar un modelo guardado en la variable `model`, y usarlo para hacer predicción con la imagen capturada.

In [ ]:
from PIL import Image
import base64
from numpy import asarray
import io
import numpy as np
import time

#pil_im = Image.open(io.BytesIO(base64.b64decode(image.split(',')[1]))) # versión en local
pil_im = Image.open(filename)  # versión en google colab
pil_im = pil_im.convert('RGB')
pil_im = pil_im.resize(IMAGE_SIZE, Image.ANTIALIAS)
pil_im

In [ ]:
im = np.asarray(pil_im) / 255
print(model.predict(np.array([im])).argmax())  # Muestra la salida del modelo y el resultado (con o sin mascarilla)
result = 'con mascarilla' if model.predict(np.array([im])).argmax() == 0 else 'sin mascarilla'
print(result)